# Storing Row and Column Attributes

## Introduction

Pilosa can store arbitrary values associated to any row or column. In Pilosa, these are referred to as `attributes`, and they can be of type `string`, `integer`, `boolean`, or `float`. In this tutorial we will store some attribute data and then run some queries that return that data.

First, create an index called `books` to use for this tutorial:

In [1]:
!curl localhost:10101/index/books \
     -X POST

{"success":true}


Next, create a field in the `books` index called `members` which will represent library members who have read books.

In [2]:
!curl localhost:10101/index/books/field/members \
     -X POST \
     -d '{}'

{"success":true}


Now, let’s add some books to our index.

In [4]:
!curl localhost:10101/index/books/query \
     -X POST \
     -d 'SetColumnAttrs(1, name="To Kill a Mockingbird", year=1960)\
         SetColumnAttrs(2, name="No Name in the Street", year=1972)\
         SetColumnAttrs(3, name="The Tipping Point", year=2000)\
         SetColumnAttrs(4, name="Out Stealing Horses", year=2003)\
         SetColumnAttrs(5, name="The Forever War", year=2008)'

{"results":[null,null,null,null,null]}


And add some members.

In [5]:
!curl localhost:10101/index/books/query \
     -X POST \
     -d 'SetRowAttrs(members, 10001, fullName="John Smith")\
         SetRowAttrs(members, 10002, fullName="Sue Perkins")\
         SetRowAttrs(members, 10003, fullName="Jennifer Hawks")\
         SetRowAttrs(members, 10004, fullName="Pedro Vazquez")\
         SetRowAttrs(members, 10005, fullName="Pat Washington")'

{"results":[null,null,null,null,null]}


At this point we can query one of the `member` records by querying that row.

In [7]:
!curl localhost:10101/index/books/query \
     -X POST \
     -d 'Row(members=10002)'

{"results":[{"attrs":{"fullName":"Sue Perkins"},"columns":[]}]}


Now let’s add some data to the matrix such that each pair represents a member who has read that book.

In [8]:
!curl localhost:10101/index/books/query \
     -X POST \
     -d 'Set(3, members=10001)\
         Set(5, members=10001)\
         Set(1, members=10002)\
         Set(2, members=10002)\
         Set(4, members=10002)\
         Set(3, members=10003)\
         Set(4, members=10004)\
         Set(5, members=10004)\
         Set(1, members=10005)\
         Set(2, members=10005)\
         Set(3, members=10005)\
         Set(4, members=10005)\
         Set(5, members=10005)'

{"results":[true,true,true,true,true,true,true,true,true,true,true,true,true]}


Now pull the record for `Sue Perkins` again.

In [9]:
!curl localhost:10101/index/books/query \
     -X POST \
     -d 'Row(members=10002)'

{"results":[{"attrs":{"fullName":"Sue Perkins"},"columns":[1,2,4]}]}


Notice that the result set now contains a list of integers in the `columns` attribute. These integers match the column IDs of the books that Sue has read.

In order to retrieve the attribute information that we stored for each book, we need to add a URL parameter `columnAttrs=true` to the query.

In [10]:
!curl localhost:10101/index/books/query?columnAttrs=true \
     -X POST \
     -d 'Row(members=10002)'

{"results":[{"attrs":{"fullName":"Sue Perkins"},"columns":[1,2,4]}],"columnAttrs":[{"id":1,"attrs":{"name":"To Kill a Mockingbird","year":1960}},{"id":2,"attrs":{"name":"No Name in the Street","year":1972}},{"id":4,"attrs":{"name":"Out Stealing Horses","year":2003}}]}


The `book` attributes are included in the result set at the `columnAttrs` attribute.

Finally, if we want to find out which books were read by both `Sue` and `Pedro`, we just perform an `Intersect` query on those two members:


In [11]:
!curl localhost:10101/index/books/query?columnAttrs=true \
     -X POST \
     -d 'Intersect(Row(members=10002), Row(members=10004))'

{"results":[{"attrs":{},"columns":[4]}],"columnAttrs":[{"id":4,"attrs":{"name":"Out Stealing Horses","year":2003}}]}


Notice that we don’t get row attributes on a complex query, but we still get the column attributes—in this case book information.